# Automate your network with Nornir – Python automation framework!

### Exploring inventory

In [ ]:
import os
os.

In [ ]:
from pprint import pprint
from colorama import Fore
import time

from nornir import InitNornir
nr = InitNornir(config_file="config.yaml")
pprint(nr.inventory.hosts)

In [ ]:
pprint(nr.inventory.groups)

### Simple output collection with netmiko

In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

results = nr.run(task=netmiko_send_command, command_string="show ip int brief | ex una")
print_result(results)

### ...or the commands parsed with TextFSM

In [ ]:
results = nr.run(task=netmiko_send_command, command_string="show version", use_textfsm=True)
print_result(results)

### Simple data retrieval using napalm

In [ ]:
from nornir.plugins.tasks.networking import napalm_get
results = nr.run(
    task=napalm_get, getters=["facts", "interfaces"]
)
print_result(results)

### Exploring connections

In [ ]:
for host in nr.inventory.hosts.values():
    print(f"{host.name} connections: {host.connections}")
    
nr.close_connections()
print(f"{Fore.RED}All connections have been closed{Fore.RESET}", end="\n\n")

for host in nr.inventory.hosts.values():
    print(f"{host.name} connections: {host.connections}")

### Data retrieval

In [ ]:
r1 = nr.inventory.hosts['R1']
print(r1.data)

In [ ]:
print(r1['tags'])  # directly from data
print(r1['ntp'])  # from group
print(r1['snmp_community'])  # from defaults
print(r1.get('non-existent-key', 'Placeholder'))  # this key does not exist in any group

### Change data dynamically

In [ ]:
# Settings site and locator for every host
for host in nr.inventory.hosts.values():
    site = host.groups[0]
    host.data['site'] = site
    locator = f'{host.name}.{site}'
    host.data['locator'] = locator

r1 = nr.inventory.hosts['R1']
print(f"{r1.name} has the following data: {r1.data}")
print(f"{r1.name} site: {r1['site']}, locator: {r1['locator']}")

### Filtering

In [ ]:
print(list(nr.filter(locator="R1.New York").inventory.hosts.keys()))
print(list(nr.filter(site="New York").inventory.hosts.keys()))

#### Advanced filtering

In [ ]:
from nornir.core.filter import F

print(list(nr.filter(F(locator="R1.New York")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London") & F(tags__contains="isr4400")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London") & F(tags__all=["isr4400", "edge"])).inventory.hosts.keys()))
print(list(nr.filter(F(ntp__servers__contains="1.2.3.4")).inventory.hosts.keys()))

### Combining filtering and task execution


In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=netmiko_send_command, command_string="show ip route")
print_result(result)

### Custom tasks

In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

def get_commands(task, commands):
    for command in commands:
        task.run(task=netmiko_send_command, command_string=command)
        
london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=get_commands, commands=["show ip int br", "show arp"])
print_result(result)
nr.close_connections()

## Building network diagram with Nornir

In [ ]:
%matplotlib inline

from typing import List, Dict, Tuple
import time

from colorama import Fore
from nornir import InitNornir
import networkx as nx
import matplotlib.pyplot as plt

from topology import parse_cdp_neighbors, build_graph

TOPOLOGY_FILENAME = "topology.png"

def draw_and_save_topology(graph: nx.Graph, edge_labels: List[Dict[Tuple[str, str], str]]) -> None:
    plt.figure(1, figsize=(12, 12))
    pos = nx.spring_layout(graph, seed=5)
    nx.draw_networkx(graph, pos, node_size=1300, node_color='orange')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels[0], label_pos=0.8)
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels[1], label_pos=0.2)
    plt.savefig(TOPOLOGY_FILENAME)
    print(f"The network topology diagram has been saved to {TOPOLOGY_FILENAME}")

start_time = time.time()
nr = InitNornir("config.yaml")
nr.run(task=parse_cdp_neighbors)
print("CDP details were successfully fetched using RESTCONF")
milestone = time.time()
time_to_run = milestone - start_time
print(f"{Fore.RED}It took {time_to_run:.2f} seconds to get and parse CDP details{Fore.RESET}")
graph, edge_labels = build_graph(nr.inventory.hosts.values())
draw_and_save_topology(graph, edge_labels)
time_to_run = time.time() - milestone
print(f"{Fore.RED}It took additional {time_to_run:.2f} seconds to draw and save the network topology{Fore.RESET}")

## END